<a href="https://colab.research.google.com/github/MatthieuHanania/PlasticDetection/blob/main/Dowloading_a_model_whithout_retrain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow==2.8.0

In [2]:
!git clone --depth 1 https://github.com/tensorflow/models

fatal: destination path 'models' already exists and is not an empty directory.


In [3]:
%%bash
cd models/research/
protoc object_detection/protos/*.proto --python_out=.
cp object_detection/packages/tf2/setup.py .
python -m pip install .

Processing /content/models/research
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached tensorflow-2.18.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.1 kB)
  Using cached tensorboard-2.18.0-py3-none-any.whl.metadata (1.6 kB)
  Using cached keras-3.6.0-py3-none-any.whl.metadata (5.8 kB)
  Using cached tensorboard_data_server-0.7.2-py3-none-manylinux_2_31_x86_64.whl.metadata (1.1 kB)
Using cached tensorflow-2.18.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (615.3 MB)
Using cached keras-3.6.0-py3-none-any.whl (1.2 MB)
Using cached tensorboard-2.18.0-py3-none-any.whl (5.5 MB)
Using cached tensorboard_data_server-0.7.2-py3-none-manylinux_2_31_x86_64.whl (6.6 MB)
  Created wheel for object_detection: filename=object_detection-0.1-py3-none-any.whl size=1697357 sha256=486d1e4cbc9258103bebd6b37d180701a7ddd700a75b8ce51f3087e7dac331ee
  Stored in directory: /tmp/pip-ephem-wheel-cache-t8zacl

In [4]:
import io
import os
import cv2
import scipy.misc
import numpy as np
import six
import time

from six import BytesIO

import matplotlib
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw, ImageFont

import tensorflow as tf
from object_detection.utils import visualization_utils as viz_utils

%matplotlib inline

In [5]:
def load_image_into_numpy_array(path):
  """Load an image from file into a numpy array.

  Puts image into numpy array to feed into tensorflow graph.
  Note that by convention we put it into a numpy array with shape
  (height, width, channels), where channels=3 for RGB.

  Args:
    path: a file path (this can be local or on colossus)

  Returns:
    uint8 numpy array with shape (img_height, img_width, 3)
  """
  img_data = tf.io.gfile.GFile(path, 'rb').read()
  image = Image.open(BytesIO(img_data))
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

# Load the COCO Label Map
category_index = {
    1: {'id': 1, 'name': 'Plastic Bottle'},

}

In [6]:
# Download a model
# model zoo : https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2_detection_zoo.md
!wget  http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_320x320_coco17_tpu-8.tar.gz
!tar -xf ssd_mobilenet_v2_320x320_coco17_tpu-8.tar.gz
!mv ssd_mobilenet_v2_320x320_coco17_tpu-8/ models/mymodel/

--2024-10-30 16:46:00--  http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_320x320_coco17_tpu-8.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 172.253.122.207, 172.253.63.207, 142.250.31.207, ...
Connecting to download.tensorflow.org (download.tensorflow.org)|172.253.122.207|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 46042990 (44M) [application/x-tar]
Saving to: ‘ssd_mobilenet_v2_320x320_coco17_tpu-8.tar.gz.1’

ssd_mobilenet_v2_32 100%[===================>]  43.91M  97.0MB/s    in 0.5s    

2024-10-30 16:46:01 (97.0 MB/s) - ‘ssd_mobilenet_v2_320x320_coco17_tpu-8.tar.gz.1’ saved [46042990/46042990]



In [7]:
tf.keras.backend.clear_session()
detect_fn = tf.saved_model.load('models/mymodel/saved_model/')

In [17]:
def export_img(image_path,image_np_with_detections, score):
    result_folder = "results/"
    image_name = "result_"+str(score)+"_"+os.path.basename(image_path)
    savepath = os.path.join(result_folder, image_name)
    print(savepath)

    cv2.imwrite(savepath, image_np_with_detections)


In [12]:
import glob

test_images_folder = 'test_images'

images = glob.glob(test_images_folder+"/*.jpg")

for image_to_test_path in images:
  image_np = load_image_into_numpy_array(image_to_test_path)
  input_tensor = np.expand_dims(image_np, 0)
  detections = detect_fn(input_tensor)

  plt.rcParams['figure.figsize'] = [42, 21]

  image_np_with_detections = image_np.copy()
  viz_utils.visualize_boxes_and_labels_on_image_array(
        image_np_with_detections,
        detections['detection_boxes'][0].numpy(),
        detections['detection_classes'][0].numpy().astype(np.int32),
        detections['detection_scores'][0].numpy(),
        category_index,
        use_normalized_coordinates=True,
        max_boxes_to_draw=200,
        min_score_thresh=.40,
        agnostic_mode=False)

  export_img(image_to_test_path,image_np_with_detections, detections['detection_scores'][0][0])

In [26]:
detections['detection_scores'][0][0]

<tf.Tensor: shape=(), dtype=float32, numpy=0.5594592690467834>

In [18]:
export_img(image_to_test_path,image_np_with_detections, detections['detection_scores'][0][0])

results/result_tf.Tensor(0.55945927, shape=(), dtype=float32)_IMG20200218112943.jpg


In [ ]:


elapsed = []
for i in range(2):
  image_path = os.path.join(image_dir, 'image' + str(i + 1) + '.jpg')


  start_time = time.time()

  end_time = time.time()
  elapsed.append(end_time - start_time)
  a = str(i)
  os.chdir(directory)
  plt.rcParams['figure.figsize'] = [42, 21]
  label_id_offset = 1
  image_np_with_detections = image_np.copy()
  viz_utils.visualize_boxes_and_labels_on_image_array(
        image_np_with_detections,
        detections['detection_boxes'][0].numpy(),
        detections['detection_classes'][0].numpy().astype(np.int32),
        detections['detection_scores'][0].numpy(),
        category_index,
        use_normalized_coordinates=True,
        max_boxes_to_draw=200,
        min_score_thresh=.40,
        agnostic_mode=False)
  #print(detections['detection_scores'][0][0].numpy(), 'llllll')
  #print(detections['detection_boxes'][0].numpy(), 'oooooooo')
  #print(detections['detection_classes'][0].numpy(), 'kkkkkk')
  plt.subplot(2, 1, i+1)
  plt.imshow(image_np_with_detections)
  export_img(image_np_with_detections, detections['detection_scores'][0][0])

mean_elapsed = sum(elapsed) / float(len(elapsed))
print('Elapsed time: ' + str(mean_elapsed) + ' second per image')